In [2]:
import sys
sys.path

['',
 '/Users/dc/miniconda3/envs/torch/lib/python36.zip',
 '/Users/dc/miniconda3/envs/torch/lib/python3.6',
 '/Users/dc/miniconda3/envs/torch/lib/python3.6/lib-dynload',
 '/Users/dc/miniconda3/envs/torch/lib/python3.6/site-packages',
 '/Users/dc/miniconda3/envs/torch/lib/python3.6/site-packages/torchvision-0.1.9-py3.6.egg',
 '/Users/dc/miniconda3/envs/torch/lib/python3.6/site-packages/IPython/extensions',
 '/Users/dc/.ipython',
 '']

In [3]:
import torch
#do not do this.. but it runs

one = torch.IntTensor(1)
five = torch.IntTensor(5)

result = one * five
print(result)

# the result is the multiplication and broadcating of 2 unitialized arrays of size 1 and 5. The result is a random result. This
#is different everytime you run it. 


 0
 0
 0
 0
 0
[torch.IntTensor of size 5]



In [2]:
import torch

#have to use numpy notation to create a tensor initialized to a value, if you do IntTensor[1] this just creates
#space for the tensor w/o values. 

one = torch.IntTensor([1])
two = torch.IntTensor([2]) 
sum = one * two
print('sum:',sum) 

#this wont run on colab.. back to the server...
#type torch.cuda.FloatTensor to get this to run on the GPU
one_gpu = torch.FloatTensor([1]).type(torch.cuda.FloatTensor)
two_gpu = torch.FloatTensor([4]).type(torch.cuda.FloatTensor)

result = one_gpu * two_gpu
print('result:',result)

sum: 
 2
[torch.IntTensor of size 1]

result: 
 4
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [5]:
# What is autograd? Based on a module autograd which calculates the gradients based on defining a DAG using foward propagation or
# the linear equation below. Once the graph is defined the autograd calculates the backprop iterations. 
# there are multiple components to autograd 
#

import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable

#define the graph with Variables=Nodes.
x = Variable(torch.Tensor([1]), requires_grad = True)
w = Variable(torch.Tensor([2]), requires_grad = True)
b = Variable(torch.Tensor([3]), requires_grad = True)

#graph edges are + and *
y = w * x + b*b

#backward() calculates the derivative WRT all variables with requires_grad=True 
y.backward()
print('x.grad: ', x.grad)
print('w.grad:', w.grad)
print('b.grad: ', b.grad)
print('-----------------')
print('x.grad.data: ', x.grad.data)
print('w.grad.data: ', w.grad.data)
print('b.grad.data: ', b.grad.data) #very cool 2*b is derivative of dy/db. 


x.grad:  Variable containing:
 2
[torch.FloatTensor of size 1]

w.grad: Variable containing:
 1
[torch.FloatTensor of size 1]

b.grad:  Variable containing:
 6
[torch.FloatTensor of size 1]

-----------------
x.grad.data:  
 2
[torch.FloatTensor of size 1]

w.grad.data:  
 1
[torch.FloatTensor of size 1]

b.grad.data:  
 6
[torch.FloatTensor of size 1]



In [11]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import os
use_cuda = torch.cuda.is_available()
print(use_cuda)


True


In [20]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    print(os.getcwd())
    #os.list(os.path.join(os.getcwd(),'data/eng-fra.txt'))
    lines = open('/home/dc/DeepLearning/torch/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
/home/dc/DeepLearning/torch
Read 135842 sentence pairs
Trimmed to 10853 sentence pairs
Counting words...
Counted words:
fra 4489
eng 2925
['tu m ennuies .', 'you re boring .']


In [21]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_output, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result     

In [22]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result


def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang, pair[0])
    target_variable = variableFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [25]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


ERROR:tornado.application:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7fae7ea63d08>)
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/tornado/ioloop.py", line 592, in _run_callback
    ret = callback()
  File "/usr/lib/python3/dist-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/IPython/kernel/zmq/ipkernel.py", line 262, in enter_eventloop
    self.eventloop(self)
  File "/usr/lib/python3/dist-packages/IPython/kernel/zmq/eventloops.py", line 151, in loop_tk
    kernel.timer = Timer(doi)
  File "/usr/lib/python3/dist-packages/IPython/kernel/zmq/eventloops.py", line 139, in __init__
    self.app = Tk()
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1871, in __init__
    self.tk = _tkinter.create(screenName, baseName, className, interactive, wantobjects, useTk, sync, use)
_tkinter.TclError: no display name and no $DISPLAY environment var

In [27]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_output, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [30]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words,
                               1, dropout_p=0.1)

if use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

trainIters(encoder1, attn_decoder1, 75000, print_every=100)

0m 8s (- 10442m 49s) (1 0%) 7.9747
0m 8s (- 5250m 11s) (2 0%) 7.9085
0m 8s (- 3525m 22s) (3 0%) 7.8888
0m 8s (- 2653m 52s) (4 0%) 2.9332
0m 8s (- 2136m 51s) (5 0%) 7.8095
0m 8s (- 1797m 48s) (6 0%) 7.8638
0m 8s (- 1553m 22s) (7 0%) 7.5794
0m 8s (- 1368m 34s) (8 0%) 3.3064
0m 8s (- 1233m 31s) (9 0%) 7.7814
0m 8s (- 1123m 55s) (10 0%) 7.6058
0m 9s (- 1036m 23s) (11 0%) 7.5798
0m 9s (- 959m 11s) (12 0%) 7.5942
0m 9s (- 891m 11s) (13 0%) 3.1201
0m 9s (- 836m 34s) (14 0%) 7.6638
0m 9s (- 786m 6s) (15 0%) 6.9348
0m 9s (- 740m 45s) (16 0%) 4.1981
0m 9s (- 701m 55s) (17 0%) 4.3298
0m 9s (- 667m 51s) (18 0%) 3.9792
0m 9s (- 636m 39s) (19 0%) 3.3565
0m 9s (- 609m 5s) (20 0%) 3.5288
0m 9s (- 583m 32s) (21 0%) 3.2057
0m 9s (- 560m 33s) (22 0%) 7.3018
0m 9s (- 539m 25s) (23 0%) 2.5519
0m 10s (- 521m 42s) (24 0%) 6.5120
0m 10s (- 504m 0s) (25 0%) 2.2865
0m 10s (- 488m 54s) (26 0%) 6.9623
0m 10s (- 473m 34s) (27 0%) 4.5783
0m 10s (- 459m 6s) (28 0%) 3.6834
0m 10s (- 446m 45s) (29 0%) 6.2199
0m 10s (-

KeyboardInterrupt: 

In [6]:
import torch
import numpy as np

VOCAB = 1000
HIDDEN = 32
SAMPLES = 2048

def logsumexp(x):
    max_x, _ = torch.max(x, dim=1, keepdim=True)
    part = torch.log(torch.sum(torch.exp(x - max_x), dim=1, keepdim=True))
    return max_x + part


l = torch.nn.Linear(HIDDEN, VOCAB)
x = torch.autograd.Variable(torch.rand(SAMPLES, HIDDEN), volatile=True)

for model_type in ['Baseline', 'MoC', 'MoS']:
    print()
    print('Testing {}'.format(model_type))
    print('==============================')

    for MIXTURES in range(1, 6):
        print('-=-=-=-=-=-=-=-=-=-=-=-')
        print('Testing for Mixture of Softmaxes (k={})'.format(MIXTURES))

        # The mixer is used in MoC and MoS
        # It takes a context vector and produces multiple of them
        mixer = torch.nn.Linear(HIDDEN, MIXTURES * HIDDEN)

        if MIXTURES == 1:
            mixed = x
        else:
            if model_type == 'Baseline':
                mixed = torch.cat([x] * MIXTURES, dim=1)
            else:
                mixed = torch.tanh(mixer(x))
        parts = torch.chunk(mixed, MIXTURES, dim=1)

        # For the different components, we're assuming equal mixing
        prior = torch.autograd.Variable(torch.ones(1, MIXTURES, 1) * (1 / MIXTURES))

        if model_type in ['Baseline', 'MoS']:
            # For each of the components, we get an entirely new full softmax result
            results = []
            for part in parts:
                results.append(torch.nn.functional.log_softmax(l(part)))
            result = torch.cat(results, dim=1).view(-1, MIXTURES, VOCAB)
            # We're in log probability space from the log_softmax, so adding the logpriors is equivalent to multiplication in prob space
            result = torch.log(prior) + result
            #print(result[0, :, :]) # Sanity check for baseline - these should be duplicate vectors

            # We're unlikely to see a difference but this compares naive logsumexp to a stable one
            result = logsumexp(result).view(-1, VOCAB)
            #naive_logsumexp_result = torch.log(torch.sum(torch.exp(result), dim=1)).view(-1, VOCAB)
            #print('Difference between naive and stable logsumexp: {}'.format((result - naive_logsumexp_result).sum().data[0]))

        elif model_type == 'MoC':
            # Mixture of Contexts is easy - just take the average of the different produced contexts
            # This makes it clearly obvious why it doesn't help with rank
            contexts = torch.cat(parts, dim=1).view(-1, MIXTURES, HIDDEN)
            moc = torch.sum(prior * contexts, dim=1)
            result = torch.nn.functional.log_softmax(l(moc))

        # Calculate the rank of the resulting matrix
        # Note: numerical precision / tolerance is important and I've not put an academic level of thought into these selections
        # Refer to proper material if you're using this within a paper
        # TODO: This is slowwwwwwwwwwwwwwwww for larger contexts ...
        rank = np.linalg.matrix_rank(result.data.numpy(), tol=1e-3)
        print('Rank of the resulting matrix (size {}): {}'.format(list(result.size()), rank))








Testing Baseline
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=1)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=2)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=3)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=4)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=5)
Rank of the resulting matrix (size [2048, 1000]): 34

Testing MoC
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=1)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=2)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixture of Softmaxes (k=3)
Rank of the resulting matrix (size [2048, 1000]): 34
-=-=-=-=-=-=-=-=-=-=-=-
Testing for Mixtu

In [9]:
import torch

m = torch.nn.Linear(20,30,bias=False)
input = torch.autograd.Variable(torch.ones(128,20))
output = m(input)
print(output.size())

torch.Size([128, 30])
